In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model

In [2]:
# Define your dataset directory
dataset_dir = 'C:\\Users\\sonaw\\Downloads\\dataset'
batch_size = 32
num_classes = 2  # T-shirt and Not T-shirt

Data Preparation and Augmentation:

ImageDataGenerator is used for data augmentation. It rescales pixel values and applies various transformations like rotation, width shift, height shift, and horizontal flip to create augmented training data.

In [3]:
# Data Augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


Loading and Preprocessing Data:

ImageDataGenerator.flow_from_directory loads and preprocesses the dataset. It specifies the target size for images and uses categorical class mode.

The flow_from_directory function loads and preprocesses the images from the specified directory (dataset_dir).

target_size sets the dimensions to which the images are resized (299x299 in this case, which is the input size for InceptionV3).

batch_size determines the number of images processed in each batch.
class_mode is set to 'categorical' since this code is prepared for multi-class classification (more than two classes).

In [4]:
# Load and preprocess your dataset
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(299, 299),  # InceptionV3's input size
    batch_size=batch_size,
    class_mode='categorical'
)

Found 1993 images belonging to 1 classes.


Loading the Pre-trained InceptionV3 Model:

The InceptionV3 model with pre-trained weights (excluding the top classification layer) is loaded from Keras' applications. This model is used as a feature extractor.

In [5]:
# Load InceptionV3 model with pre-trained weights (excluding top classification layer)
base_model = InceptionV3(weights='imagenet', include_top=False)

Custom Classification Layers:

Custom classification layers are added on top of the pre-trained InceptionV3 model. These include global average pooling, a dense layer with 1024 units, and a final dense layer with 2 units (for binary classification) and softmax activation.

In [6]:
# Add your custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

Freezing Pre-trained Layers:

The pre-trained layers of InceptionV3 are set to non-trainable to retain the knowledge from the pre-trained model.

This loop iterates through the layers of the pre-trained InceptionV3 model and sets them to be non-trainable. This is a common practice to retain the pre-trained knowledge and prevent it from being updated during training.

In [7]:

# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

Model Compilation:

The model is compiled using the Adam optimizer, categorical cross-entropy loss (suitable for multiple classes), and accuracy as the evaluation metric. Since you are performing binary classification, you may consider changing the loss function to 'binary_crossentropy' and the output layer to have a single unit with sigmoid activation.

In [8]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

The model is compiled with the Adam optimizer, categorical cross-entropy loss (suitable for multi-class classification), and accuracy as the evaluation metric. If you are performing binary classification, you may want to change the loss function to 'binary_crossentropy' and modify the output layer accordingly.

Model Training:

The model is trained using the augmented data for a specified number of epochs. You can adjust the number of epochs as needed.

In [10]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=1,  # You can adjust the number of epochs
)

63/63 [==============================] - 541s 9s/step - loss: 29.5077 - accuracy: 0.5168


The model is trained using the training data generator (train_generator) for a specified number of epochs. You can adjust the number of epochs as needed for training.

In [11]:

# Save the model
model.save('tshirt_classifier.h5')

C:\Users\sonaw\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


The trained model is saved to a file named 'tshirt_classifier.h5' for future use, such as making predictions on new data.

In [13]:
# Define the test data directory
test_data_dir = 'C:\\Users\\sonaw\\Downloads\\dataset'

# Create a test data generator
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(299, 299),  # Match the input size of InceptionV3
    batch_size=batch_size,
    class_mode='categorical'
)

# Make predictions on the test data
predictions = model.predict(test_generator)



Found 1993 images belonging to 1 classes.
63/63 [==============================] - 439s 7s/step


model.fit is a function used to train a neural network model. It takes several parameters, and here's what each parameter does:

train_generator: This is the data generator that provides batches of training data to the model. In your code, train_generator is an instance of ImageDataGenerator that reads and augments images from the 'dataset_dir'. It's used to generate training data in batches.

steps_per_epoch: This parameter determines how many batches of data are processed in each epoch. len(train_generator) calculates the number of batches available in the train_generator. Setting steps_per_epoch to this value means that the model will train on the entire dataset (all batches) in one epoch.

epochs: An epoch is one complete pass through the entire training dataset. In this code, you've set epochs to 1, which means the model will go through all the available training data once. You can adjust this number to control how many times the model should iterate over the entire dataset.

In [14]:

# Evaluate the model
eval_metrics = model.evaluate(test_generator)
print(f'Accuracy: {eval_metrics[1]}')

63/63 [==============================] - 424s 7s/step - loss: 4.8117 - accuracy: 0.9408
Accuracy: 0.9407927989959717


The accuracy is approximately 94.08%.